In [46]:
using MLDatasets
using Distances
using Statistics
using Clustering
using Printf
using DataStructures
using Random
using SumProductNetworks

In [13]:
train_x, train_y = MNIST.traindata()
train_x = MNIST.convert2features(train_x)
train_2 = train_x[:, train_y .== 2]

784×5958 Array{N0f8,2} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0

In [14]:
println(size(train_2))
println(typeof(train_2))

(784, 5958)
Array{FixedPointNumbers.Normed{UInt8,8},2}


In [98]:
function learnspn_c(data; minclustersize=100)
    """A minimal implementation of Learn-SPN algorithm
    
    Repeat sum node and product node in turn to form binary SPNs.
    """
    q = Queue{Tuple}()
    root = FiniteSumNode()
    variables = collect(1:size(data)[1])
    instances = collect(1:size(data)[2])
    enqueue!(q, (root, variables, instances))
    
    while length(q) > 0
        node, variables, instances = dequeue!(q)
        
        # stopping condition 1: only one variable is remaining
        if length(variables) == 1
            prod = FiniteProductNode()
            v = variables[1]
            μ_hat = mean(data[v, :])
            σ_hat = std(data[v, :]) + 0.01
            add!(prod, UnivariateNode(Normal(μ_hat, σ_hat), v))
            add!(node, prod, log(1.0))
            continue
        end
        
        # stopping condition 2: instance cluster is two small
        if length(instances) < minclustersize
            for v in variables
                μ_hat = mean(data[v, :])
                σ_hat = std(data[v, :]) + 0.01
                add!(node, UnivariateNode(Normal(μ_hat, σ_hat), v))
            end
            continue
        end
        
        # divide and conquer
        if typeof(node) <: SumNode
            clusterweights = clusterinstances(data, variables, instances)
            for (cluster, weight) in clusterweights
                prod = FiniteProductNode()
                add!(node, prod, log(weight))
                enqueue!(q, (prod, variables, cluster))
            end
        elseif typeof(node) <: ProductNode
            splits = splitvariables(data, variables, instances)
            for split in splits
                sum = FiniteSumNode()
                add!(node, sum)
                enqueue!(q, (sum, split, instances))
            end
        else
            error("encountered unknown node type")
        end
    end
    
    return SumProductNetwork(root)
end

function splitvariables(data, variables, instances)
    """Split variables into two groups
    
    Run pseudo G-test (KL-divergence from observation to expectation).
    """
    function binarize(x)
        binary_x = zeros(Int, size(x))
        binary_x[x .> mean(x)] .= 1
        return binary_x
    end
    slice = data[variables, instances]
    distances = zeros(length(variables))
    p = sum(binarize(slice[rand(1:length(variables)), :]))/length(instances)
    for i in 1:length(variables)
        q = sum(binarize(slice[i, :]))/length(instances)
        e = (p + q)/2
        d = evaluate(KLDivergence(),
                     [p, (1 - p), q, (1 - q)],
                     [e, (1 - e), e, (1 - e)])
        distances[i] = d
    end
    dependentindex = partialsortperm(distances, 1:floor(Integer, length(variables)/2))
    splitone = variables[dependentindex]
    splittwo = setdiff(variables, splitone)
    return (splitone, splittwo)
end

function clusterinstances(data, variables, instances)
    """Cluster instances into two groups
    
    Run K-means clustering with k=2.
    """
    slice = data[variables, instances]
    results = kmeans(slice, 2)
    clusterone = instances[results.assignments .== 1]
    clustertwo = setdiff(instances, clusterone)
    weight = length(clusterone)/length(instances)
    return ((clusterone, weight), (clustertwo, 1 - weight))
end

clusterinstances (generic function with 1 method)

In [99]:
spn = learnspn_c(train_2)

SumProductNetwork
	#nodes = 68610
	#leaves = 46579
	depth = 23


In [93]:
updatescope!(spn)

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

...

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/pa

Excessive output truncated after 524520 bytes.┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:48
┌ Warning: New scope is larger than original scope, resize node scope...
└

FiniteSumNode{Float64}(##sum#148419)
	weights = [0.566633, 0.433367]
	normalized = true
	scope = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 

In [94]:
llhvals = initllhvals(spn, train_2[:, 1])
logpdf!(spn, train_2[:, 1], llhvals)
for node in spn.leaves
    println("logpdf at $(node.id) = $(llhvals[node.id])")
end

logpdf at ##univ#199450 = 3.6862316527834182
logpdf at ##univ#209210 = 3.6862316527834182
logpdf at ##univ#209212 = 3.6862316527834182
logpdf at ##univ#204199 = 3.6862316527834182
logpdf at ##univ#204200 = 3.6862316527834182
logpdf at ##univ#192706 = 3.6862316527834182
logpdf at ##univ#192707 = 3.6862316527834182
logpdf at ##univ#192708 = 3.6862316527834182
logpdf at ##univ#199454 = 3.6862316527834182
logpdf at ##univ#209214 = 3.6862316527834182
logpdf at ##univ#209216 = 3.6862316527834182
logpdf at ##univ#204203 = 3.6862316527834182
logpdf at ##univ#204204 = 3.6862316527834182
logpdf at ##univ#192711 = 3.6862316527834182
logpdf at ##univ#192712 = 3.6862316527834182
logpdf at ##univ#192713 = 3.6862316527834182
logpdf at ##univ#183053 = 3.6862316527834182
logpdf at ##univ#183054 = 3.6862316527834182
logpdf at ##univ#183055 = 3.6862316527834182
logpdf at ##univ#183056 = 3.6862316527834182
logpdf at ##univ#183057 = 3.6862316527834182
logpdf at ##univ#183058 = 3.6862316527834182
logpdf at 

logpdf at ##univ#204263 = 3.6862316527834182
logpdf at ##univ#204264 = 3.6862316527834182
logpdf at ##univ#192781 = 3.6862316527834182
logpdf at ##univ#192782 = 3.6862316527834182
logpdf at ##univ#192783 = 3.6862316527834182
logpdf at ##univ#192784 = 3.6862316527834182
logpdf at ##univ#183109 = 3.6862316527834182
logpdf at ##univ#183110 = 3.6862316527834182
logpdf at ##univ#183111 = 3.6862316527834182
logpdf at ##univ#183112 = 3.6862316527834182
logpdf at ##univ#183113 = 3.6862316527834182
logpdf at ##univ#183114 = 3.6862316527834182
logpdf at ##univ#183115 = 3.6862316527834182
logpdf at ##univ#169855 = 3.6862316527834182
logpdf at ##univ#169856 = 3.6862316527834182
logpdf at ##univ#169857 = 3.6862316527834182
logpdf at ##univ#169858 = 3.6862316527834182
logpdf at ##univ#169859 = 3.6862316527834182
logpdf at ##univ#169860 = 3.6862316527834182
logpdf at ##univ#169861 = 3.6862316527834182
logpdf at ##univ#169862 = 3.6862316527834182
logpdf at ##univ#169863 = 3.6862316527834182
logpdf at 

logpdf at ##univ#156609 = 3.6862316527834182
logpdf at ##univ#156610 = 3.3340548344920067
logpdf at ##univ#156611 = 3.6862316527834182
logpdf at ##univ#156612 = 3.6862316527834182
logpdf at ##univ#156613 = 3.6862316527834182
logpdf at ##univ#156614 = 3.6862316527834182
logpdf at ##univ#156615 = 3.6862316527834182
logpdf at ##univ#156616 = 3.6862316527834182
logpdf at ##univ#199546 = 3.6862316527834182
logpdf at ##univ#209310 = 3.6862316527834182
logpdf at ##univ#209312 = 3.6862316527834182
logpdf at ##univ#204299 = 3.6862316527834182
logpdf at ##univ#204300 = 3.6862316527834182
logpdf at ##univ#192827 = 3.6862316527834182
logpdf at ##univ#192828 = 3.6862316527834182
logpdf at ##univ#192829 = 3.6862316527834182
logpdf at ##univ#199550 = 3.389477911194478
logpdf at ##univ#209314 = 3.6862316527834182
logpdf at ##univ#209316 = 3.6862316527834182
logpdf at ##univ#204303 = 3.6862316527834182
logpdf at ##univ#204304 = 3.6862316527834182
logpdf at ##univ#192832 = 3.389477911194478
logpdf at ##

logpdf at ##univ#150188 = 3.6862316527834182
logpdf at ##univ#150189 = 3.6862316527834182
logpdf at ##univ#150190 = 3.6862316527834182
logpdf at ##univ#150191 = 3.6862316527834182
logpdf at ##univ#150192 = 3.6862316527834182
logpdf at ##univ#150193 = 3.6862316527834182
logpdf at ##univ#150194 = 3.6862316527834182
logpdf at ##univ#150195 = 3.6862316527834182
logpdf at ##univ#150196 = 3.6862316527834182
logpdf at ##univ#150197 = 3.6862316527834182
logpdf at ##univ#150198 = 3.6862316527834182
logpdf at ##univ#150199 = 3.6862316527834182
logpdf at ##univ#150200 = 3.6862316527834182
logpdf at ##univ#150201 = 3.6862316527834182
logpdf at ##univ#150202 = 3.6862316527834182
logpdf at ##univ#150203 = 3.6862316527834182
logpdf at ##univ#150204 = 3.6862316527834182
logpdf at ##univ#150205 = 3.6862316527834182
logpdf at ##univ#150206 = 3.6862316527834182
logpdf at ##univ#150207 = 3.6862316527834182
logpdf at ##univ#150208 = 3.6862316527834182
logpdf at ##univ#150209 = 3.6270511497182416
logpdf at 

logpdf at ##univ#209386 = 3.6862316527834182
logpdf at ##univ#209388 = 3.6862316527834182
logpdf at ##univ#204375 = 3.6862316527834182
logpdf at ##univ#204376 = 3.6862316527834182
logpdf at ##univ#192918 = 3.6862316527834182
logpdf at ##univ#192919 = 3.6862316527834182
logpdf at ##univ#192920 = 3.6862316527834182
logpdf at ##univ#199622 = 3.348779878003892
logpdf at ##univ#209390 = 2.583338717180778
logpdf at ##univ#209392 = 3.6862316527834182
logpdf at ##univ#204379 = 2.583338717180778
logpdf at ##univ#204380 = 3.6862316527834182
logpdf at ##univ#192923 = 3.348779878003892
logpdf at ##univ#192924 = 2.583338717180778
logpdf at ##univ#192925 = 3.6862316527834182
logpdf at ##univ#183223 = 3.6862316527834182
logpdf at ##univ#183224 = 3.6862316527834182
logpdf at ##univ#183225 = 3.6862316527834182
logpdf at ##univ#183226 = 3.348779878003892
logpdf at ##univ#183227 = 2.583338717180778
logpdf at ##univ#183228 = 3.6862316527834182
logpdf at ##univ#169955 = 3.6862316527834182
logpdf at ##univ#

...

logpdf at ##univ#194179 = 0.017581427807948358
logpdf at ##univ#194180 = 0.09315504607937775
logpdf at ##univ#194181 = 1.1507382691759311
logpdf at ##univ#200498 = 1.1845760015071798
logpdf at ##univ#210274 = 1.2159482740019798
logpdf at ##univ#210276 = 1.298907880715905
logpdf at ##univ#205303 = 1.2159482740019798
logpdf at ##univ#205304 = 1.298907880715905
logpdf at ##univ#194184 = 1.1845760015071798
logpdf at ##univ#194185 = 1.2159482740019798
logpdf at ##univ#194186 = 1.298907880715905
logpdf at ##univ#184803 = 1.1507382691759311
logpdf at ##univ#184804 = 1.1845760015071798
logpdf at ##univ#184805 = 1.2159482740019798
logpdf at ##univ#184806 = 1.298907880715905
logpdf at ##univ#184807 = 0.09315504607937775
logpdf at ##univ#184808 = 0.017581427807948358
logpdf at ##univ#173762 = 0.7659672036513573
logpdf at ##univ#173763 = 0.7062500664596911
logpdf at ##univ#173764 = 0.7674167619356549
logpdf at ##univ#173765 = 0.8655645439076011
logpdf at ##univ#173766 = 0.9258365456062095
logpdf a

logpdf at ##univ#152460 = 0.12951115590516815
logpdf at ##univ#152461 = -1.316889664765121
logpdf at ##univ#152462 = 0.563292399666215
logpdf at ##univ#152463 = -2.084674061381236
logpdf at ##univ#152464 = 0.23498367863052305
logpdf at ##univ#152465 = -0.9266390181343112
logpdf at ##univ#152466 = 0.3995784250956542
logpdf at ##univ#152467 = 0.6269410284616533
logpdf at ##univ#152468 = 0.06410154481851005
logpdf at ##univ#152469 = -1.3300525282681437
logpdf at ##univ#152470 = 0.5041173139635432
logpdf at ##univ#152471 = 0.4614009657066632
logpdf at ##univ#152472 = -0.06979979504281297
logpdf at ##univ#152473 = -1.056405931986558
logpdf at ##univ#152474 = 0.2654550925074475
logpdf at ##univ#152475 = -0.1670835574498033
logpdf at ##univ#152476 = -0.08230859187921458
logpdf at ##univ#152477 = -2.5981295925991965
logpdf at ##univ#152478 = 0.14541071490004454
logpdf at ##univ#152479 = -2.0878409354727117
logpdf at ##univ#152480 = 0.48388538290003946
logpdf at ##univ#152481 = 0.48869176665553

logpdf at ##univ#152684 = -1.5112282752626012
logpdf at ##univ#152685 = 0.335594101740055
logpdf at ##univ#152686 = -1.509369752276999
logpdf at ##univ#152687 = 0.5944375296108456
logpdf at ##univ#152688 = -0.09096448459868534
logpdf at ##univ#152689 = 0.42223550075868044
logpdf at ##univ#152690 = -1.316889664765121
logpdf at ##univ#152691 = -1.4348344614077497
logpdf at ##univ#152692 = 0.28122241177783425
logpdf at ##univ#152693 = 0.0992464837824456
logpdf at ##univ#152694 = -0.08230859187921458
logpdf at ##univ#152695 = -0.18369140560827446
logpdf at ##univ#152696 = 0.6269410284616533
logpdf at ##univ#152697 = -0.09198381309685544
logpdf at ##univ#152698 = -3.3771771305019844
logpdf at ##univ#152699 = 0.4278149935630844
logpdf at ##univ#158262 = -0.7472559794463743
logpdf at ##univ#158263 = -0.5330235729373993
logpdf at ##univ#158264 = 0.017642042931035107
logpdf at ##univ#158265 = -2.0878409354727117
logpdf at ##univ#158266 = -2.084674061381236
logpdf at ##univ#158267 = -0.027821786

Excessive output truncated after 524314 bytes.

logpdf at ##univ#173838 = 0.6269410284616533
logpdf at ##univ#173839 = 0.5944375296108456
logpdf at ##univ#173840 = 0.48869176665553515
logpdf at ##univ#173841 = 0.42223550075868044
logpdf at ##univ#173842 = 0.0992464837824456
logpdf at ##univ#173843 = 0.335594101740055
logpdf at ##univ#173844 = 0.4278149935630844
logpdf at ##univ#184890 = 0.28122241177783425
logpdf at ##univ#184891 = -0.18369140560827446
logpdf at ##univ#184892 = -0.08230859187921458
logpdf at ##univ#184893 = 0.0635482276844137
logpdf at ##univ#184894 = 0.0026469107292539373
logpdf at ##univ#184895 = -1.4348344614077497
logpdf at ##univ#184896 = 0.28122241177783425


In [88]:
logp = logpdf(spn, train_2[:, 1])

1053.697231402501

In [89]:
logp = logpdf(spn, train_x[:, 10])

746.7819958258569

In [100]:
# Create a root sum node.
root = FiniteSumNode();

# Add two product nodes to the root.
add!(root, FiniteProductNode(), log(0.3)); # Use a weight of 0.3
add!(root, FiniteProductNode(), log(0.7)); # Use a weight of 0.7

# Add Normal distributions to the product nodes, i.e. leaves.
for prod in children(root)
    for d in 1:2 # Assume 2-D data
        add!(prod, UnivariateNode(Normal(), d));
    end
end

# Compile the constructed network to an SPN type
spn = SumProductNetwork(root);

# Print statistics on the network.
println(spn)

# Update the scope of all nodes, i.e. propagate the scope bottom-up.
updatescope!(spn)

# Evaluate the network on some data.
x = [0.8, 1.2];
logp = logpdf(spn, x)
println(logp)

# Access the stored log likelihood
llh = spn.info[:llh]
println(llh)

# Evaluate the network by marginalising out a RV.
x = [0.8, NaN]
logp = logpdf(spn, x)
println(logp)

SumProductNetwork
	#nodes = 7
	#leaves = 4
	depth = 3

-2.8778770664093463
-2.8778770664093463
-1.2389385332046732


┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
┌ Warning: New scope is larger than original scope, resize node scope...
└ @ SumProductNetworks /home/kouariga/.julia/packages/SumProductNetworks/8eDzv/src/nodeFunctions.jl:34
